In [1]:
import csv
from cobra.io import read_sbml_model, write_sbml_model
from copy import copy
from dotenv import find_dotenv
import os
from os.path import dirname

In [2]:
MODEL_PATH = "models/yeast-GEM.xml" # Lu, H., Li, F., Sánchez, B.J. et al. A consensus S. cerevisiae metabolic model Yeast8 and its ecosystem for comprehensively probing cellular metabolism. Nat Commun 10, 3586 (2019). https://doi.org/10.1038/s41467-019-11581-3

In [3]:
from functions import *

In [4]:
model = read_yeast_model(MODEL_PATH) # loading

In [5]:
write_yeast_model(model, "model_yeast8.xml")   # saving

# The Yeast8 model at a glance

In [6]:
model.medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0}

In [7]:
media_comp = list(model.medium)
for i in range (0, len(media_comp)):
    print(model.medium[media_comp[i]], model.reactions.get_by_id(media_comp[i]).name)
    if 'glucose' in model.reactions.get_by_id(media_comp[i]).name:
        carbon_exchange_id = media_comp[i]
        glucose_id = model.reactions.get_by_id(carbon_exchange_id).reaction[:8] # carbon source ID

1000.0 ammonium exchange
1.0 D-glucose exchange
1000.0 H+ exchange
1000.0 iron(2+) exchange
1000.0 oxygen exchange
1000.0 phosphate exchange
1000.0 potassium exchange
1000.0 sodium exchange
1000.0 sulphate exchange
1000.0 water exchange
1000.0 chloride exchange
1000.0 Cu2(+) exchange
1000.0 Mn(2+) exchange
1000.0 Zn(2+) exchange
1000.0 Mg(2+) exchange
1000.0 Ca(2+) exchange


In [8]:
print("The Yeast8 Saccharomyces cerevisiae model contains "+str(len(model.genes))+" genes, "+str(len(model.metabolites))+" metabolites and "+str(len(model.reactions))+" reactions.")

The Yeast8 Saccharomyces cerevisiae model contains 1150 genes, 2742 metabolites and 4058 reactions.


In [9]:
print(model.compartments)

{'ce': 'cell envelope', 'c': 'cytoplasm', 'e': 'extracellular', 'm': 'mitochondrion', 'n': 'nucleus', 'x': 'peroxisome', 'r': 'endoplasmic reticulum', 'g': 'Golgi', 'lp': 'lipid particle', 'v': 'vacuole', 'rm': 'endoplasmic reticulum membrane', 'vm': 'vacuolar membrane', 'gm': 'Golgi membrane', 'mm': 'mitochondrial membrane'}


In [10]:
for metabolite in model.metabolites.query('glucose', 'name'):
    print(metabolite.name,metabolite.id)

D-glucose glc__D_c
D-glucose glc__D_e
D-glucose glc__D_v
D-glucose 1-phosphate g1p_c
D-glucose 6-phosphate g6p_c
UDP-D-glucose udpg_c
D-glucose glc__D_r
2-deoxy-D-glucose 6-phosphate 2doxg6p_c
2-deoxy-D-glucose 2dglc_c
UDP-D-glucose udpg_r
D-glucose 1-phosphate g1p_e
D-glucose 6-phosphate g6p_e


In [11]:
print("The growth rate of Yeast8 is: "+str(model.optimize().objective_value)[:5])
biomass_id = str(model.objective).split(' ')[0].split('*')[1]

The growth rate of Yeast8 is: 0.083


In [12]:
for reactions in model.reactions.query('ferrochelatase', 'name'):
    print(reactions.name,reactions.id)
    ferrochelatase_id = str(reactions.id)
model.reactions.get_by_id(ferrochelatase_id)

ferrochelatase FCLTm


Reaction identifier,FCLTm
Name,ferrochelatase
Memory address,0x02af9a100688
Stoichiometry,fe2_m + ppp9_m --> 2.0 h_m + pheme_m iron(2+) + protoporphyrin --> 2.0 H+ + ferroheme b
GPR,YOR176W
Lower bound,0.0
Upper bound,1000.0


In [13]:
medium = model.medium
with model:
    solution = model.optimize()
    print("Yeast8 maximum theoretical biomass productivity:", solution.fluxes[biomass_id], '/h')
    model.medium = medium
    model.objective = model.reactions.get_by_id(ferrochelatase_id)
    heme_production = model.optimize().objective_value
    print("Yeast8t maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

Yeast8 maximum theoretical biomass productivity: 0.08374778664999834 /h
Yeast8t maximum theoretical productivity of heme  8.374770489917409e-08 [mmol gDW^-1 h^-1]


## Enumerated pathway to heme

In [14]:
heme_1 = model.reactions.get_by_id('ALASm')
heme_2 = model.reactions.get_by_id('PPBNGS')
heme_3 = model.reactions.get_by_id('HMBS')
heme_4 = model.reactions.get_by_id('UPP3S') 
heme_12 = model.reactions.get_by_id('UPPDC1') 
heme_13 = model.reactions.get_by_id('CPPPGO')
heme_14 = model.reactions.get_by_id('PPPGOm') # second to last reaction in the heme biosynthesis
heme_15 = model.reactions.get_by_id('FCLTm')
heme_pathway = [heme_1,heme_2,heme_3,heme_4,heme_12,heme_13,heme_14,heme_15]

In [15]:
for i in range (0,len(heme_pathway)):
    with model:
        model.objective = heme_pathway[-1] # calculates the flux with heme as the objective
        print(model.optimize().fluxes[heme_pathway[i].id], heme_pathway[i].id, heme_pathway[i].reaction, heme_pathway[i].name)

6.699816391933927e-07 ALASm gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 5-aminolevulinate synthase
3.3499081959669635e-07 PPBNGS 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c porphobilinogen synthase
8.374770489917409e-08 HMBS h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c hydroxymethylbilane synthase
8.374770489917409e-08 UPP3S hmbil_c --> h2o_c + uppg3_c uroporphyrinogen-III synthase
8.374770489917409e-08 UPPDC1 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c uroporphyrinogen decarboxylase (uroporphyrinogen III)
8.374770489917409e-08 CPPPGO cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c coproporphyrinogen oxidase (O2 required)
4.1873852449587043e-08 PPPGOm 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m protoporphyrinogen oxidase
8.374770489917409e-08 FCLTm fe2_m + ppp9_m --> 2.0 h_m + pheme_m ferrochelatase


## Pseudopathway for recombinant haemoglobin

In [29]:
from functions import *
from cobra import Reaction, Metabolite

In [17]:
add_rhb(model, "model_yeast8_rhb.xml")

In [30]:
MODEL_PATH = "model_yeast8_rhb.xml"
model = read_yeast_model(MODEL_PATH) # loading

## Enumerated pathway to recombinant haemoglobin in yeast, starting with glycine

In [31]:
rhb_pathway = get_rhb_pathway(model)

In [32]:
print(rhb_pathway) # now we can work with the model in other notebooks (i.e. media, mutants, etc.)

[<Reaction ALASm at 0x2af9e80fa48>, <Reaction PPBNGS at 0x2af9d071488>, <Reaction HMBS at 0x2af9d1a0ec8>, <Reaction UPP3S at 0x2af9d0fc188>, <Reaction UPPDC1 at 0x2af9d0ee7c8>, <Reaction CPPPGO at 0x2af9cff4388>, <Reaction PPPGOm at 0x2af9d08a308>, <Reaction FCLTm at 0x2af9e0dd508>, <Reaction rHb at 0x2afa2da5988>]


In [33]:
for i in range (0,len(rhb_pathway)):
    with model:
        model.objective = rhb_pathway[-1] # calculates the flux with heme as the objective
        print(model.optimize().fluxes[rhb_pathway[i].id], rhb_pathway[i].id, rhb_pathway[i].reaction, rhb_pathway[i].name)

0.051054370293057864 ALASm gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 5-aminolevulinate synthase
0.025527185146528932 PPBNGS 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c porphobilinogen synthase
0.006381796286632233 HMBS h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c hydroxymethylbilane synthase
0.006381796286632233 UPP3S hmbil_c --> h2o_c + uppg3_c uroporphyrinogen-III synthase
0.006381796286632233 UPPDC1 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c uroporphyrinogen decarboxylase (uroporphyrinogen III)
0.006381796286632233 CPPPGO cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c coproporphyrinogen oxidase (O2 required)
0.0031908981433161165 PPPGOm 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m protoporphyrinogen oxidase
0.006381796286632233 FCLTm fe2_m + ppp9_m --> 2.0 h_m + pheme_m ferrochelatase
0.0015954490716580583 rHb 72.0 ala__L_c + 12.0 arg__L_c + 20.0 asn__L_c + 30.0 asp__L_c + 6.0 cys__L_c + 8.0 gln__L_c + 24.0 glu__L_c + 40.0 gly_c + 38.0 his__L_c + 72.0 leu__L_

In [39]:
medium = model.medium
with model:
    model.medium = medium
    model.objective = model.reactions.rHb
    rhb_production = model.optimize().objective_value
    print("Max. recombinant haemoglobin production: "+str(rhb_production)[:6]+" [mmol gDW^-1 h^-1]")
    print("Theoretical max. yield: "+ str(rhb_production / (-1*model.reactions.EX_glc__D_e.flux))[:6]+" [mmol-rHb / mmol-glucose]")

Max. recombinant haemoglobin production: 0.0015 [mmol gDW^-1 h^-1]
Theoretical max. yield: 0.0015 [mmol-rHb / mmol-glucose]


# TODO
# OptReg: High-throughput calculations of knock-outs, up- or downregulations.

MEWpy: https://github.com/BioSystemsUM/mewpy

Vítor Pereira, Fernando Cruz, Miguel Rocha, MEWpy: a computational strain optimization workbench in Python, Bioinformatics, 2021;, btab013, https://doi.org/10.1093/bioinformatics/btab013

# Rationale-based genetic modifications

#### Up-regulation

HEM2	YGL040C,
HEM3	YDL205C,
HEM13	YDR044W,
HEM15	YOR176W

#### Knockout

HAP1	YLR256W

#### Downregulation
YFH1	YDL120W

#### Maybe interesting
CCC1	YLR220W	Transport af heme - into vacule


PUG1	YER185W	Export af heme - ud af cellen

	
YPL172C knockout


YAL039C knockout

In [40]:
for gene in model.genes.query('YFH1', 'name'):
    print(gene.name,gene.id)

YFH1 YDL120W


In [42]:
for gene in model.genes.query('HAP4', 'name'):
    print(gene.name,gene.id)